<style type="text/css">
#container { width: 100px; //whatever width you want }

#image {width: 100%; //fill up whole div }

#text { text-align: justify; }    
</style>

 <div id="container"> 
     <img src="https://fambus.org/wp-content/uploads/2016/02/usd-logo.png" id="image" /> 
     <h3 id="text">Leonardo F. de Souza</h3>
     <h3 id="text">FIN 414 - Financial Derivatives</h3>
     <h3 id="text">Dr. James Driver</h3>
     <h3 id="text">Binomial Option Pricing Model</h3>
 </div>

________

In [116]:
import numpy as np
import datetime as dt
import yfinance as yf
import pandas as pd
import pandas_datareader.data as web
from scipy.stats import norm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash_bootstrap_templates import load_figure_template
import warnings
from scipy.interpolate import griddata
warnings.filterwarnings('ignore')
layout = load_figure_template('DARKLY')

1. Inputs

In [30]:
S0 = 100
K = 100
r = 0.02
N = 2
u = 1.1
d = 0.9
div = 0
T = 1
h = T/(N-1)
re = np.exp(r*h)
q = ((re-d)/(u-d))

print(f'Spot: ${round(S0,3)}')
print(f'Strike: ${round(K,3)}')
print(f'Rates: {round(r*100,3)}%')
print(f'Number of Steps: {round(N,3)}')
print(f'Time to maturity: {round(T,3)} year')
print(f'Up-Factor: {round(u,3)}')
print(f'Down-Factor: {round(d,3)}')
print(f'Dividend Yield: {round(div,3)}')
print(f'Upward Probability: {round(q*100,3)}%')

Spot: $100
Strike: $100
Rates: 2.0%
Number of Steps: 2
Time to maturity: 1 year
Up-Factor: 1.1
Down-Factor: 0.9
Dividend Yield: 0
Upward Probability: 60.101%


## Generic Binomial Tree

In [31]:
def binomial_model(N, S0, u, d, r, K, q, CP, EA):
    if EA == 'E':

        stock = np.zeros([N + 1, N + 1])
        for i in range(N + 1):
            for j in range(i + 1):
                stock[j, i] = S0 * (u ** (i - j)) * (d ** j)
                
        option = np.zeros([N + 1, N + 1])
        if CP == 'P':
            option[:, N] = np.maximum(np.zeros(N + 1), (K - stock[:, N]))
        elif CP == 'C':
            option[:, N] = np.maximum(np.zeros(N + 1), (stock[:, N] - K))
        for i in range(N - 1, -1, -1):
            for j in range(0, i + 1):
                option[j, i] = ((q * option[j, i + 1] + (1-q) * option[j + 1, i + 1]))/re
        return option
    elif EA == 'A':
        
        stock = np.zeros(N+1)
        for j in range(0, N+1):
            stock[j] = S0 * (u**j) * (d**(N-j))
            
        option = np.zeros(N+1)
        for j in range(0, N+1):
            if CP == 'P':
                option[j] = max(0, K - stock[j])
            elif CP == 'C':
                option[j] = max(0, stock[j] - K)
                
        for i in np.arange(N-1,-1,-1):
            for j in range(0,i+1):
                stock = S0 * (u**j) * (d**(i-j))
                option[j] = (q*option[j+1] + (1-q)*option[j])/re
                if CP == 'P':
                    option[j] = max(option[j], K - stock)
                elif CP == 'C':
                    option[j] = max(option[j], stock - K)
                
        return option[0]
        

print(f"Call Price (European): ${round(binomial_model(N, S0, u, d, r, K, q, CP = 'C', EA='E')[0,0], 2)}")
print(f"Put Price (European): ${round(binomial_model(N, S0, u, d, r, K, q, CP = 'P', EA='E')[0,0], 2)}")
print()
print(f"Call Price (American): ${round(binomial_model(N, S0, u, d, r, K, q, CP = 'C', EA='A'),2)}")
print(f"Put Price (American): ${round(binomial_model(N, S0, u, d, r, K, q, CP = 'P', EA='A'),2)}")

Call Price (European): $7.29
Put Price (European): $3.37

Call Price (American): $7.29
Put Price (American): $4.14


________

## Lognormal Distribution

We might have two main frameworks (or solutions) to use (though, there might be multiple other ways to implement this. Just a though... but could RSI, MACD, or Bollinger Bands have something to do with expected returns and/or volatility? Coming from a momentum perspective).

We will evaluate the option price based on Cox-Ross-Rubinstein's (CRR) and Jarrow-Rudd's (JR) propositions.

<div>
    <p> CRR: </p>
    </div>
\begin{align}
u_{CRR} & = e^{\sigma\sqrt{h}}\label{eq:CRR}\tag{1}\\
d_{CRR} & = e^{-\sigma\sqrt{h}} = \frac{1}{u_{CRR}}\tag{2} \\
p_{CRR} & = \frac{1}{2} + \frac{\mu}{2\sigma}\sqrt{h}\tag{3}
\end{align}


<div>
    <p> JR: </p>
    </div>
\begin{align}
u_{JR} & = e^{\mu h + \sigma\sqrt{h}} \label{eq:JR}\tag{4}\\
d_{JR} & = e^{\mu h - \sigma\sqrt{h}}\tag{5}\\
p_{JR} & = \frac{1}{2} \tag{6}
\end{align}

Where $h$:

\begin{align}
h & = \frac{T}{N} \label{eq:h}\tag{7}
\end{align}

$T$ is the time to maturity in years<br>
$N$ is the number of steps in the binomial tree

The probability $p$ still follows the formula:
\begin{align}
p & = \frac{e^{r\frac{T}{N}} - d}{u-d} = \frac{e^{rh} - d}{u-d} \label{eq:p}\tag{8}\\
\end{align}

________

- For this example, let's use IJR (iShares Core S&P Small-Cap ETF)
- $T$ = 1 year
- $N$ = 100 steps
- Dates: 11/1/2015 - 11/1/2022

In [223]:
Ticker = 'IJR'
T = 1
N = 100
h = T/N
div = 0

end = dt.datetime(2022,11,1)
start = dt.datetime(2015,11,1)
interval = '1mo'

df = yf.download(Ticker, interval=interval, start=start)['Adj Close']
df = pd.DataFrame(df)
df = df.dropna()
df1 = np.log(df) - np.log(df.shift(1))

df1 = df1[1:-1:]

[*********************100%***********************]  1 of 1 completed


- Company's Name (for charts)

In [224]:
ticker = yf.Ticker(Ticker)
inf = ticker.info
Name = pd.DataFrame.from_dict(inf, orient="index").T
Name = Name[["shortName"]]
Name = str(Name["shortName"].values[0])
print(f'Name: {Name}')

Name: iShares Core S&P Small-Cap ETF


- Rolling Returns and Standard Deviation:

In [225]:
Ret_Rol = df1.rolling(12).mean().dropna()
Std_Rol = df1.rolling(12).std().dropna()

figure = make_subplots(rows=2, cols=1, subplot_titles=('Returns', 'Standard Deviation'))
figure.add_trace(go.Scatter(x=Ret_Rol.index, y=Ret_Rol['Adj Close'], name='Returns'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Std_Rol.index, y=Std_Rol['Adj Close'], name='Stdev'), row = 2, col = 1)
figure.update_layout(xaxis1=dict(showgrid=False),
                  yaxis1=dict(showgrid=False),
                  xaxis2=dict(showgrid=False),
                  yaxis2=dict(showgrid=False))
figure.update_layout(
            xaxis2=dict(rangeselector=dict(buttons=list([
                dict(count=1,label="1Y",step="year",stepmode="backward"),
                dict(count=5,label="5Y",step="year",stepmode="backward"),
                dict(count=1,label="YTD",step="year",stepmode="todate"),
                dict(step="all")])),
                        rangeslider=dict(visible=False),
                        type="date"))
figure.update_layout(
            xaxis1=dict(rangeselector=dict(buttons=list([
                dict(count=1,label="1Y",step="year",stepmode="backward"),
                dict(count=5,label="5Y",step="year",stepmode="backward"),
                dict(count=1,label="YTD",step="year",stepmode="todate"),
                dict(step="all")])),
                        rangeslider=dict(visible=False),
                        type="date"))
figure.update_layout(yaxis_tickformat=',.2%', yaxis2_tickformat=',.2%')
figure.update_layout(template='plotly_dark',
                  xaxis_rangeselector_font_color='black',
                  xaxis_rangeselector_activecolor='gold',
                  xaxis_rangeselector_bgcolor='grey',
                  xaxis2_rangeselector_font_color='black',
                  xaxis2_rangeselector_activecolor='gold',
                  xaxis2_rangeselector_bgcolor='grey')
figure.update_layout(title_text= Name+" 1-Year Rolling", title_x=0.5)

- Let "r" be the Market Yield on U.S. Treasury Securities at 1-Year Constant Maturity, Quoted on an Investment Basis

In [235]:
Rates = 'DGS1' #FRED

Rates = web.DataReader(Rates, 'fred', start, end)
Rates = Rates.dropna()
Rates = Rates['DGS1'].values[-1]/100 # as %

r = Rates
re = np.exp(r*h)
print(f'Rates: {round(r*100,4)}%')

Rates: 4.75%


- The annualized standard deviation of monthly returns and the (geometric) average of returns is given as:

\begin{align}
\sigma_{Annualized} & = \sigma_{Monthly}\sqrt{12} \label{eq:std}\tag{9}\\
CAGR & = (\prod_{i=1}^{n} x_{i})^{\frac{1}{n}} = 
 \left[(1+r_{1})(1+r_{2})...(1+r_{n})\right]^{\frac{12}{n}}-1 \label{eq:CAGR}\tag{10}
\end{align}

In [236]:
std = df1['Adj Close'].std()*np.sqrt(12)
avg = ((np.prod(1+df1['Adj Close']))**(12/((df1['Adj Close'].count()))))-1

print(f"g: {round(avg*100,2)}%")
print(f"σ: {round(std*100,2)}%")

g: 7.13%
σ: 21.7%


- CRR:

In [237]:
u_c = np.exp(std * np.sqrt(h))
d_c = 1/u_c
p_c = 0.5 + 0.5*(avg/std)*np.sqrt(h)
q_c = (re-d_c)/(u_c-d_c)

print(f'u: {round(u_c,3)}')
print(f'd: {round(d_c,3)}')
print(f'p: {round(p_c,3)}')
print(f'q: {round(q_c,3)}')

u: 1.022
d: 0.979
p: 0.516
q: 0.506


- JR:

In [238]:
u_j = np.exp((avg*h) + (std*np.sqrt(h)))
d_j = np.exp((avg*h) - (std*np.sqrt(h)))
p_j = 1/2
q_j = (re-d_j)/(u_j-d_j)

print(f'u: {round(u_j,3)}')
print(f'd: {round(d_j,3)}')
print(f'p: {round(p_j,3)}')
print(f'q: {round(q_j,3)}')

u: 1.023
d: 0.979
p: 0.5
q: 0.489


- Parameters for the generic model:

In [239]:
S0 = df['Adj Close'].values[-1]
K = int(1*np.ceil(df['Adj Close'].values[-1]/1)) # Round-up to closest ($5 step option)
r = Rates
re = re
N = 100
div = 0

print(f'Spot: ${round(S0,3)}')
print(f'Strike: ${round(K,3)}.00')
print(f'Rates: {round(Rates*100,3)}0%')
print(f'Number of Steps: {round(N,3)}')

Spot: $100.9
Strike: $101.00
Rates: 4.750%
Number of Steps: 100


1. CRR:

In [240]:
print(f"Call Price (European): ${round(binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='E')[0,0], 2)}")
print(f"Put Price (European): ${round(binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='E')[0,0], 2)}")
print()
print(f"Call Price (American): ${round(binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='A'),2)}")
print(f"Put Price (American): ${round(binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='A'),2)}")

Call Price (European): $10.99
Put Price (European): $6.4

Call Price (American): $10.99
Put Price (American): $6.91


2. JR:

In [241]:
print(f"Call Price (European): ${round(binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='E')[0,0], 2)}")
print(f"Put Price (European): ${round(binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='E')[0,0], 2)}")
print()
print(f"Call Price (American): ${round(binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='A'),2)}")
print(f"Put Price (American): ${round(binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='A'),2)}")

Call Price (European): $11.02
Put Price (European): $6.44

Call Price (American): $11.02
Put Price (American): $6.93


3. Black-Scholes (BS):

\begin{align}
Call_{BS} & = S_{0}N(d_{1}) - Ke^{-rT}N(d_{2}) \label{eq:BSC}\tag{11}\\
Put_{BS} & = Ke^{-rT}N(d_{2}) - S_{0}N(d_{1}) \label{eq:BSP}\tag{12}\\
\end{align}

Where $d_{1}$ and $d_{2}$:
\begin{align}
d_{1} & = \frac{\ln({\frac{S_{0}}{K}}) + (r+\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}} \label{eq:d1}\tag{13}\\
d_{2} & = \frac{\ln({\frac{S_{0}}{K}}) + (r-\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}} = d_{1} - \sigma\sqrt{T} \label{eq:d2}\tag{14}\\
\end{align}

In [242]:
def BSM(r, S0, K, T, std, type):
    d1 = (np.log(S0/K) + (r + ((std**2)/2))*T)/(std*np.sqrt(T))
    d2 = d1 - std*np.sqrt(T)
    try:
        if type == "C":
            P = S0*norm.cdf(d1, 0, 1) - K*(np.exp(-r*T))*norm.cdf(d2, 0, 1)
        elif type == 'P':
            P = K*(np.exp(-r*T))*norm.cdf(-d2, 0, 1) - S0*norm.cdf(-d1, 0, 1)
        return round(P, 2)
    except:
        print("Error")
        
print(f"Call Price: ${(BSM(r, S0, K, T, std, type='C'))}")
print(f"Put Price: ${(BSM(r, S0, K, T, std, type='P'))}")

Call Price: $11.0
Put Price: $6.42


- Models vs. Number of Steps:

In [243]:
Ns = np.linspace(1,200,200).astype(int)
CRRC = []
FRC = []
CRRP = []
FRP = []

CRRC_A = []
FRC_A = []
CRRP_A = []
FRP_A = []

for n in Ns:
    T = 1
    h = T/n
    re = np.exp(r*h)
    u_c = np.exp(std * np.sqrt(h))
    d_c = 1/u_c
    p_c = 0.5 + 0.5*(avg/std)*np.sqrt(h)
    q_c = (re-d_c)/(u_c-d_c)
    u_j = np.exp((avg*h) + (std*np.sqrt(h)))
    d_j = np.exp((avg*h) - (std*np.sqrt(h)))
    p_j = 1/2
    q_j = (re-d_j)/(u_j-d_j)
    
    # European
    CRR1 = binomial_model(n, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='E')[0,0]
    FR1 = binomial_model(n, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='E')[0,0]
    CRR2 = binomial_model(n, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='E')[0,0]
    FR2 = binomial_model(n, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='E')[0,0]
    
    # American
    CRR3 = binomial_model(n, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='A')
    FR3 = binomial_model(n, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='A')
    CRR4 = binomial_model(n, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='A')
    FR4 = binomial_model(n, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='A')
    
    # European
    CRRC.append(CRR1)
    FRC.append(FR1)
    CRRP.append(CRR2)
    FRP.append(FR2)
    
    # American
    CRRC_A.append(CRR3)
    FRC_A.append(FR3)
    CRRP_A.append(CRR4)
    FRP_A.append(FR4)

Ns = np.array(Ns)

CRRC = np.array(CRRC)
FRC = np.array(FRC)
CRRP = np.array(CRRP)
FRP = np.array(FRP)

CRRC_A = np.array(CRRC_A)
FRC_A = np.array(FRC_A)
CRRP_A = np.array(CRRP_A)
FRP_A = np.array(FRP_A)

Opt = [Ns, CRRC, FRC, CRRP, FRP, CRRC_A, FRC_A, CRRP_A,  FRP_A]
Opt = pd.DataFrame(Opt).T
Opt.columns = ['Steps', 'CRRC', 'FRC', 'CRRP', 'FRP', 'CRRC_A', 'FRC_A', 'CRRP_A', 'FRP_A']
Opt['BSC'] = BSM(r, S0, K, T, std, type='C')
Opt['BSP'] = BSM(r, S0, K, T, std, type='P')

figure = make_subplots(rows=2, cols=2, subplot_titles=('European Call', 'European Put', 'American Call', 'American Put'))
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['CRRC'], legendgroup="Euro Call",
                 legendgrouptitle_text="Euro Call", name='CRR'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['FRC'], legendgroup="Euro Call", name='FR'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['BSC'], legendgroup="Euro Call", name='BS'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['CRRP'], legendgroup="Euro Put",
                 legendgrouptitle_text="Euro Put", name='CRR'), row = 1, col = 2)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['FRP'], legendgroup="Euro Put", name='FR'), row = 1, col = 2)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['BSP'], legendgroup="Euro Put", name='BS'), row = 1, col = 2)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['CRRC_A'], legendgroup="American Call",
                 legendgrouptitle_text="American Call", name='CRR'), row = 2, col = 1)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['FRC_A'], legendgroup="American Call", name='FR'), row = 2, col = 1)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['BSC'], legendgroup="American Call", name='BS'), row = 2, col = 1)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['CRRP_A'],legendgroup="American Put",
                 legendgrouptitle_text="American Put", name='CRR'), row = 2, col = 2)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['FRP_A'], legendgroup="American Put", name='FR'), row = 2, col = 2)
figure.add_trace(go.Scatter(x=Opt['Steps'], y=Opt['BSP'], legendgroup="American Put", name='BS'), row = 2, col = 2)
figure.update_layout(yaxis1_tickprefix = '$', yaxis2_tickprefix =  '$', 
                     yaxis3_tickprefix =  '$',yaxis4_tickprefix =  '$',
                     yaxis1_tickformat = ',.2f',yaxis2_tickformat = ',.2f',
                     yaxis3_tickformat = ',.2f', yaxis4_tickformat = ',.2f')
figure.update_layout(title_text= Name +": Option Premia on a 1-Year $"+str(round(K,3))+ 
                     " Strike Contract, by Number of Steps",
                     yaxis1_title ='Price($)', yaxis3_title ='Price($)', title_x=0.5 )
figure.update_layout(yaxis1=dict(showgrid=False),yaxis2=dict(showgrid=False),
                     yaxis3=dict(showgrid=False),yaxis4=dict(showgrid=False),
                     xaxis1=dict(showgrid=False), xaxis2=dict(showgrid=False),
                     xaxis3=dict(showgrid=False),xaxis4=dict(showgrid=False),)
figure.update_layout(template='plotly_dark')
figure.update_layout(legend=dict(groupclick="toggleitem"))
#figure.update_layout(legend=dict(orientation="h"))

- Payoff

In [244]:
returns1 = np.linspace(0, 2*S0,200)
returns1
aa = pd.DataFrame(returns1)
aa.columns = ['St']
aa['Put'] = np.where(aa['St'] <= K, K-aa['St'],0)
aa['Call'] = np.where(aa['St']>=K, aa['St']-K, 0)
aa['NPut_BS'] = aa['Put'] - BSM(r, S0, K, T, std, type='P')
aa['NPut_CRR'] = aa['Put'] - binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='E')[0,0]
aa['NPut_JR'] = aa['Put'] - binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='E')[0,0]
aa['NCall_BS'] = aa['Call'] - BSM(r, S0, K, T, std, type='C')
aa['NCall_CRR'] = aa['Call'] - binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='E')[0,0]
aa['NCall_JR'] = aa['Call'] - binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='E')[0,0]

aa['NPut_CRRA'] = aa['Put'] - binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='A')
aa['NPut_JRA'] = aa['Put'] - binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='A')

fig = make_subplots(rows=1, cols=2, subplot_titles=('American', 'European'), shared_yaxes='all')
fig.add_trace(go.Scatter(x=aa['St'], y=aa['Put'], legendgroup="European",
                 legendgrouptitle_text="European", name= 'Gross'), row=1, col=2)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['NPut_BS'], legendgroup="European", name = 'BS'), row=1, col=2)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['NPut_CRR'], legendgroup="European", name = 'CRR'), row=1, col=2)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['NPut_JR'], legendgroup="European", name = 'JR'), row=1, col=2)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['Put'], legendgroup="American",
                 legendgrouptitle_text="American", name= 'Gross'), row=1, col=1)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['NPut_BS'], legendgroup="American", name = 'BS'), row=1, col=1)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['NPut_CRRA'], legendgroup="American", name = 'CRR'), row=1, col=1)
fig.add_trace(go.Scatter(x=aa['St'], y=aa['NPut_JRA'], legendgroup="American", name = 'JR'), row=1, col=1)
fig.update_layout(yaxis1_tickprefix = '$', yaxis1_tickformat = ',.2f',
                  xaxis1_tickprefix = '$', xaxis1_tickformat = ',.2f')
fig.update_layout(yaxis2_tickprefix = '$', yaxis2_tickformat = ',.2f',
                  xaxis2_tickprefix = '$', xaxis2_tickformat = ',.2f')
fig.update_layout(xaxis1=dict(showgrid=False),
                  yaxis1=dict(showgrid=False),
                  xaxis2=dict(showgrid=False),
                  yaxis2=dict(showgrid=False))
fig.update_layout(title_text= Name + ": American vs. European Long-Put",
                  yaxis_title ='Profit($)',
                  xaxis_title = 'St', xaxis2_title='St', title_x=0.5)
fig.update_layout(template='plotly_dark')
fig.update_layout(legend=dict(groupclick="toggleitem"))
# fig.update_layout(legend=dict(orientation="h"))

- Strike vs. Option Prices:

In [245]:
Ks = np.linspace(1,200,200).astype(int)

CRRC_E = []
FRC_E = []
CRRP_E = []
FRP_E = []

BSC1 = []
BSP1 = []

CRRC_A = []
FRC_A = []
CRRP_A = []
FRP_A = []

for K in Ks:
    
    # European
    CRR1 = binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='E')[0,0]
    FR1 = binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='E')[0,0]
    CRR2 = binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='E')[0,0]
    FR2 = binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='E')[0,0]
    
    # American
    CRR3 = binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='A')
    FR3 = binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'C', EA='A')
    CRR4 = binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'P', EA='A')
    FR4 = binomial_model(N, S0, u_j,d_j, re, K, q_j, CP = 'P', EA='A')
    
    # BS
    BSC = BSM(r, S0, K, T, std, type='C')
    BSP = BSM(r, S0, K, T, std, type='P')
    
    # European
    CRRC_E.append(CRR1)
    FRC_E.append(FR1)
    CRRP_E.append(CRR2)
    FRP_E.append(FR2)
    
    # American
    CRRC_A.append(CRR3)
    FRC_A.append(FR3)
    CRRP_A.append(CRR4)
    FRP_A.append(FR4)
    
    # BS
    BSC1.append(BSC)
    BSP1.append(BSP)

Ks = np.array(Ks)

CRRC_E = np.array(CRRC_E)
FRC_E = np.array(FRC_E)
CRRP_E = np.array(CRRP_E)
FRP_E = np.array(FRP_E)

CRRC_A = np.array(CRRC_A)
FRC_A = np.array(FRC_A)
CRRP_A = np.array(CRRP_A)
FRP_A = np.array(FRP_A)

BSC1 = np.array(BSC1)
BSP1 = np.array(BSP1)

Stk = [Ks, CRRC_E, FRC_E, CRRP_E, FRP_E, CRRC_A, FRC_A, CRRP_A, FRP_A, BSC1, BSP1]
Stk = pd.DataFrame(Stk).T
Stk.columns = ['Steps', 'CRRC', 'FRC', 'CRRP', 'FRP', 'CRRC_A', 'FRC_A', 'CRRP_A', 'FRP_A', 'BSC', 'BSP']

figure = make_subplots(rows=2, cols=2, subplot_titles=('European Call', 'European Put', 'American Call', 'American Put'))
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['CRRC'], legendgroup="Euro Call",
                 legendgrouptitle_text="Euro Call", name='CRR'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['FRC'], legendgroup="Euro Call", name='FR'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['BSC'], legendgroup="Euro Call", name='BS'), row = 1, col = 1)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['CRRP'], legendgroup="Euro Put",
                 legendgrouptitle_text="Euro Put", name='CRR'), row = 1, col = 2)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['FRP'], legendgroup="Euro Put", name='FR'), row = 1, col = 2)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['BSP'], legendgroup="Euro Put", name='BS'), row = 1, col = 2)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['CRRC_A'], legendgroup="American Call",
                 legendgrouptitle_text="American Call", name='CRR'), row = 2, col = 1)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['FRC_A'], legendgroup="American Call", name='FR'), row = 2, col = 1)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['BSC'], legendgroup="American Call", name='BS'), row = 2, col = 1)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['CRRP_A'],legendgroup="American Put",
                 legendgrouptitle_text="American Put", name='CRR'), row = 2, col = 2)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['FRP_A'], legendgroup="American Put", name='FR'), row = 2, col = 2)
figure.add_trace(go.Scatter(x=Stk['Steps'], y=Stk['BSP'], legendgroup="American Put", name='BS'), row = 2, col = 2)
figure.update_layout(yaxis1_tickprefix = '$', yaxis2_tickprefix =  '$', 
                     yaxis3_tickprefix =  '$',yaxis4_tickprefix =  '$',
                     yaxis1_tickformat = ',.2f',yaxis2_tickformat = ',.2f',
                     yaxis3_tickformat = ',.2f', yaxis4_tickformat = ',.2f')
figure.update_layout(xaxis1_tickprefix = '$', xaxis2_tickprefix =  '$', 
                     xaxis3_tickprefix =  '$',xaxis4_tickprefix =  '$',
                     xaxis1_tickformat = ',.2f',xaxis2_tickformat = ',.2f',
                     xaxis3_tickformat = ',.2f', xaxis4_tickformat = ',.2f')
figure.update_layout(title_text= Name +": Strike vs. Premia, For Constant T=1",
                     yaxis1_title ='Price($)', yaxis3_title ='Price($)', title_x=0.5 )
figure.update_layout(yaxis1=dict(showgrid=False),yaxis2=dict(showgrid=False),
                     yaxis3=dict(showgrid=False),yaxis4=dict(showgrid=False),
                     xaxis1=dict(showgrid=False), xaxis2=dict(showgrid=False),
                     xaxis3=dict(showgrid=False),xaxis4=dict(showgrid=False),)
figure.update_layout(template='plotly_dark')
figure.update_layout(legend=dict(groupclick="toggleitem"))
#figure.update_layout(legend=dict(orientation="h"))

- We could also look at the relationship Price (P) vs. Strike (K) vs. Time to Expiration (T):

In [246]:
Ts = np.linspace(0,1,50)
Ks = np.linspace(S0-20,S0+20,40).astype(int)

Call = []
Time = []
Strike =[]

for T in Ts:
    for K in Ks:
        T = T
        N = 100
        h = T/N
        re = np.exp(r*h)
        u_c = np.exp(std * np.sqrt(h))
        d_c = 1/u_c
        p_c = 0.5 + 0.5*(avg/std)*np.sqrt(h)
        q_c = (re-d_c)/(u_c-d_c)
        u_j = np.exp((avg*h) + (std*np.sqrt(h)))
        d_j = np.exp((avg*h) - (std*np.sqrt(h)))
        p_j = 1/2
        q_j = (re-d_j)/(u_j-d_j)
        EC = binomial_model(N, S0, u_c,d_c, re, K, q_c, CP = 'C', EA='E')[0,0]
    
        Call.append(EC)
        Time.append(T)
        Strike.append(K)
        

Ks = np.array(Strike)
Ts = np.array(Time)
Call = np.array(Call)

PKT = [Ks, Ts, Call]
PKT = pd.DataFrame(PKT).T
PKT.columns = ['K', 'T', 'O']
PKT = Pika.dropna()

In [247]:
# xi = PKT['T'].tolist()
# yi = PKT['K'].tolist()

# x_grid, y_grid = np.meshgrid(xi,yi)

# #Grid data
# z_grid = griddata((PKT['T'],PKT['K']),PKT['O'],(x_grid,y_grid),method='cubic')

# # Plotly 3D Surface
# fig = go.Figure(go.Surface(x=x_grid,y=y_grid,z=z_grid,
#                            colorscale='viridis',showlegend=False))
# fig.update_layout(scene = dict(
#                     xaxis_title='Time to Maturity (T)',
#                     yaxis_title='Strike (K)',
#                     zaxis_title='Premium (P)'))
# fig.update_layout(title_text= Name +" Call: Premium vs. Strike vs. Time")
# fig.update_layout(yaxis_tickprefix =  '$', yaxis_tickformat = ',.2f')

# fig.show()

_____
## Option Greeks
###### Black-Scholes

1. Delta ($\Delta$) measures the rate of change of the theoretical option value with respect to changes in the underlying asset's price.
\begin{align}
\Delta & = \frac{\partial V}{\partial S} \label{eq:delt} \tag{15}\\
\end{align}
<br>
\begin{align}
Call \Delta = e^{-qT}\Phi(d_{1})\tag{16}\\
Put \Delta = -e^{-qT}\Phi(-d_{1})\tag{17}\\
\end{align}
<br>
2. Theta ($\Theta$) measures the sensitivity of the value of the derivative to the passage of time, or time decay.
\begin{align}
\Theta & = \frac{\partial V}{\partial T}\tag{18}\\
\end{align}
<br>
\begin{align}
Call \Theta = -e^{-qT}\frac{S\phi(d_{1})\sigma}{2\sqrt{T}} - rKe^{-rT}\Phi(d_{2}) + qSe^{-qT}\Phi(d_{1})\tag{19}\\
Put \Theta = -e^{-qT}\frac{S\phi(d_{1})\sigma}{2\sqrt{T}} + rKe^{-rT}\Phi(-d_{2}) - qSe^{-qT}\Phi(-d_{1})\tag{20}\\
\end{align}
<br>
3. Rho ($\rho$) measures sensitivity to the interest rate: it is the derivative of the option value with respect to the risk free interest rate.
\begin{align}
\rho & = \frac{\partial V}{\partial r}\tag{21}\\
\end{align}
<br>
\begin{align}
Call \rho = KTe^{-rT}\Phi(d_{2})\tag{22}\\
Put \rho = -KTe^{-rT}\Phi(-d_{2})\tag{23}\\
\end{align}
<br>
4. Vega ($\nu$) measures sensitivity to volatility. Vega is the derivative of the option value with respect to the volatility of the underlying asset.
\begin{align}
\nu & = \frac{\partial V}{\partial \sigma}\tag{24}\\
\end{align}
<br>
\begin{align}
\nu = Se^{-qT}\phi(d_{1})\sqrt{T} = Ke^{-rT}\phi(d_{2})\sqrt{T}\label{eq:vega}\tag{25}\\
\end{align}
<br>
5. Gamma ($\Gamma$) measures the rate of change in the delta with respect to changes in the underlying price.
\begin{align}
\Gamma & = \frac{\partial \Delta}{\partial S} = \frac{\partial^{2} V}{\partial S^{2}} \tag{26}\\
\end{align}
<br>
\begin{align}
\Gamma = e^{-qT}\frac{\phi(d_{1})}{S\sigma\sqrt{T}} = Ke^{-rT}\frac{\phi(d_{2})}{S^{2}\sigma\sqrt{T}}\label{eq:gamma}\tag{27}\\
\end{align}
<br>
Where:
\begin{align}
d_{1} & = \frac{\ln({\frac{S_{0}}{K}}) + (r+\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}}\\
d_{2} & = \frac{\ln({\frac{S_{0}}{K}}) + (r-\frac{\sigma^{2}}{2})T}{\sigma\sqrt{T}} = d_{1} - \sigma\sqrt{T}\\
\phi(x) & = \frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2}x^{2}} \tag{28}\\
\Phi(x) & = \frac{1}{\sqrt{2\pi}}\int_{x}^{-\infty} e^{-\frac{1}{2}y^{2}} dy = 1 - \frac{1}{\sqrt{2\pi}}\int_{\infty}^{x} e^{-\frac{1}{2}y^{2}} dy\tag{29}\\
\end{align}
<br>
$d_{1}$ and $d_{2}$ were previously defined by equations $(13)$ and $(14)$ 
<br>
<br>
- Note that $\nu$ $\eqref{eq:vega}$ and $\Gamma$ $\eqref{eq:gamma}$ have the same value for calls and puts.
<br>
<br>
<br>
Back to the pricing model, consider the following parameters:

In [248]:
S0 = df['Adj Close'].values[-1]
K = int(1*np.ceil(df['Adj Close'].values[-1]/1)) # Round-up to closest ($5 step option)
r = Rates
re = re
T = 1
N = 100
div = 0
std = std

print(f'Spot: ${round(S0,3)}')
print(f'Strike: ${round(K,3)}.00')
print(f'Rates: {round(Rates,3)*100}0%')
print(f'Number of Steps: {round(N,3)}')
print(f'Time to Maturity (T): {round(T,3)} year')
print(f'Standard Deviation (σ): {round(std,3)*100}%')

Spot: $100.9
Strike: $101.00
Rates: 4.80%
Number of Steps: 100
Time to Maturity (T): 1 year
Standard Deviation (σ): 21.7%


Let's define $d_{1}$ and $d_{2}$:

In [249]:
def ds(std, S0, K, r, T):
    d1 = (np.log(S0/K) + (r + ((std**2)/2))*T)/(std*np.sqrt(T))
    d2 = d1 - std*np.sqrt(T)
    return d1, d2

d1 = ds(std, S0, K, r, T)[0]
d2 = ds(std, S0, K, r, T)[1]
print(f'd1: {round(d1,3)}')
print(f'd2: {round(d2,3)}')

d1: 0.323
d2: 0.106


Calculations for $\Delta$, $\Theta$, $\rho$, $\nu$, and $\Gamma$:
- For these calculations, assume dividend yield ($q$) as zero.

In [250]:
def delta(d1, CP):
    if CP == 'C':
        return norm.cdf(d1)
    elif CP == 'P':
        return -norm.cdf(-d1)
def theta(S0, std, d1, T, r, d2, CP):
    if CP == 'C':
        return - S0*std*norm.pdf(d1)/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2)
    elif CP == 'P':
        return - S0*std*norm.pdf(d1)/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2)
def rho(K, T, r, d2, CP):
    if CP == 'C':
        return K*T*np.exp(-r*T)*norm.cdf(d2)
    elif CP == 'P':
        return -K*T*np.exp(-r*T)*norm.cdf(-d2)
def vega(K, r, T, d2):
    return K*np.exp(-r*T)*norm.pdf(d2)*np.sqrt(T)
def gamma(K, r, T, d2, S0, std):
    return K*np.exp(-r*T)*(norm.pdf(d2)/((S0**2)*std*np.sqrt(T)))

delta = [round(delta(d1, CP='C'),3), round(delta(d1, CP='P'),3)]
theta = [round(theta(S0, std, d1, T, r, d2, CP='C'), 3), round(theta(S0, std, d1, T, r, d2, CP='P'),3)]
rho = [round(rho(K, T, r, d2, CP='C'),3), round(rho(K, T, r, d2, CP='P'),3)]
vega = [round(vega(K, r, T, d2),3), round(vega(K, r, T, d2),3)]
gamma = [round(gamma(K, r, T, d2, S0, std),3), round(gamma(K, r, T, d2, S0, std),3)]

Greeks = [delta, theta, rho, vega, gamma]
Greeks = pd.DataFrame(Greeks)
Greeks.columns = ['Call', 'Put']
Greeks.insert(0,'Greeks',['Δ', 'Θ', '𝜌', '𝜈', 'Γ'])
# Greeks = Greeks.style.hide_index()
Greeks

,Greeks,Call,Put
0,Δ,0.627,-0.373
1,Θ,-6.625,-2.050
2,𝜌,52.219,-44.096
3,𝜈,38.209,38.209
4,Γ,0.017,0.017


___
### Generalized AutoRegressive Conditional Heteroskedasticity (GARCH)